<a href="https://www.kaggle.com/code/muradpitafi/bostan-house-price-using-advance-regression?scriptVersionId=154766006" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-boston-houseprice-data/boston.csv


In [2]:
bostan = pd.read_csv('/kaggle/input/the-boston-houseprice-data/boston.csv')
bostan.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [3]:
bostan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(12), int64(2)
memory usage: 55.5 KB


In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
import warnings
warnings.filterwarnings('ignore')
data = bostan.drop(['MEDV'], axis = 1)
for i in data.columns:
    data[i] = scaler.fit_transform(data[[i]])

In [5]:
from sklearn.model_selection import train_test_split
x = data
y = bostan['MEDV']

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [6]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
lr = LinearRegression()
lr.fit(xtrain, ytrain)
print('training ', lr.score(xtrain, ytrain))
print('testing ', lr.score(xtest, ytest))
ypred = lr.predict(xtest)
print('mse ', mean_squared_error(ytest, ypred))
print('rmse', np.sqrt(mean_squared_error(ytest, ypred)))

training  0.7730135569264233
testing  0.589222384918251
mse  33.44897999767652
rmse 5.783509315085134


In [7]:
lasso = Lasso(alpha=1.0)
lasso.fit(xtrain, ytrain)
print('training ', lasso.score(xtrain, ytrain))
print('testing ', lasso.score(xtest, ytest))
ypred = lasso.predict(xtest)
print('mse ', mean_squared_error(ytest, ypred))
print('rmse', np.sqrt(mean_squared_error(ytest, ypred)))

training  0.7062017176361302
testing  0.504490101495281
mse  40.348597574959406
rmse 6.352054594771633


In [8]:
ridge = Ridge(alpha = 1.0)
ridge.fit(xtrain, ytrain)
print('training ', ridge.score(xtrain, ytrain))
print('testing ', ridge.score(xtest, ytest))
ypred = ridge.predict(xtest)
print('mse ', mean_squared_error(ytest, ypred))
print('rmse', np.sqrt(mean_squared_error(ytest, ypred)))

training  0.7729987527288801
testing  0.5881439261558414
mse  33.53679721129215
rmse 5.791096373856349


In [9]:
from sklearn.model_selection import cross_val_score

# Linear Regression
lr = LinearRegression()
cv_scores = cross_val_score(lr, xtrain, ytrain, cv=5, scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(-cv_scores)
 
print('Cross-Validation RMSE:', cv_rmse)
print('Mean CV RMSE:', np.mean(cv_rmse))


Cross-Validation RMSE: [4.01835181 4.59619826 4.50419069 5.62626391 4.08417755]
Mean CV RMSE: 4.5658364433145


In [10]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Lasso Regression
lasso = Lasso()
lasso_grid = GridSearchCV(lasso, param_grid, cv=5, scoring='neg_mean_squared_error')
lasso_grid.fit(xtrain, ytrain)


# Ridge Regression
ridge = Ridge()
ridge_grid = GridSearchCV(ridge, param_grid, cv=5, scoring='neg_mean_squared_error')
ridge_grid.fit(xtrain, ytrain)

# Display best hyperparameters
print('Best Lasso Alpha:', lasso_grid.best_params_['alpha'])
print('Best Ridge Alpha:', ridge_grid.best_params_['alpha'])


Best Lasso Alpha: 0.01
Best Ridge Alpha: 10


In [11]:
ridge = Ridge(alpha = 10)
ridge.fit(xtrain, ytrain)
print('training ', ridge.score(xtrain, ytrain))
print('testing ', ridge.score(xtest, ytest))
ypred = ridge.predict(xtest)
print('mse ', mean_squared_error(ytest, ypred))
print('rmse', np.sqrt(mean_squared_error(ytest, ypred)))

training  0.7720285977838584
testing  0.5791883293835864
mse  34.266037477324666
rmse 5.8537199691584725


In [12]:
lasso = Lasso(alpha=0.01)
lasso.fit(xtrain, ytrain)
print('training ', lasso.score(xtrain, ytrain))
print('testing ', lasso.score(xtest, ytest))
ypred = lasso.predict(xtest)
print('mse ', mean_squared_error(ytest, ypred))
print('rmse', np.sqrt(mean_squared_error(ytest, ypred)))
print('r2 ', r2_score(ytest, ypred))

training  0.7729551595655116
testing  0.5874766938690956
mse  33.59112889489356
rmse 5.79578544244812
r2  0.5874766938690956


In [13]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)
xpoly = poly.fit_transform(xtrain)
testpoly = poly.transform(xtest)

lr.fit(xpoly, ytrain)

pred = lr.predict(testpoly)

print('r2 score ', r2_score(ytest, pred))



r2 score  0.6117641811910122


In [14]:
lr.coef_
lr.intercept_


819302349551.9171

In [15]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb

In [16]:
rfc = RandomForestRegressor(n_estimators=100)
rfc.fit(xtrain, ytrain)
ypred = rfc.predict(xtest)
print('mse ', mean_squared_error(ytest, ypred))
print('rmse', np.sqrt(mean_squared_error(ytest, ypred)))
print('r2 ', r2_score(ytest, ypred))
 

mse  18.89151430392158
rmse 4.34643696651885
r2  0.7679985700135936


In [17]:
gbr = GradientBoostingRegressor()
gbr.fit(xtrain, ytrain)
ypred = gbr.predict(xtest)

print('mse ', mean_squared_error(ytest, ypred))
print('rmse', np.sqrt(mean_squared_error(ytest, ypred)))
print('r2 ', r2_score(ytest, ypred))
 

mse  17.831047639223527
rmse 4.222682516981774
r2  0.7810218660133109


In [18]:
xg = xgb.XGBRegressor()
xg.fit(xtrain,ytrain)
ypred = xg.predict(xtest)
print('mse ', mean_squared_error(ytest, ypred))
print('rmse', np.sqrt(mean_squared_error(ytest, ypred)))
print('r2 ', r2_score(ytest, ypred))
 

mse  21.780087522670343
rmse 4.666914132772355
r2  0.7325248061538571


In [19]:
import numpy as np

def linear_regression(X, y, learning_rate=0.01, num_iterations=1000):
    # Add a column of ones to the feature matrix for the bias term
    X = np.column_stack((np.ones(len(X)), X))
    
    num_samples, num_features = X.shape

    # Initialize weights and bias
    weights = np.zeros(num_features)
    bias = 0

    # Gradient Descent
    for _ in range(num_iterations):
        # Calculate predictions
        y_pred = np.dot(X, weights) + bias

        # Calculate gradients
        dw = (1/num_samples) * np.dot(X.T, (y_pred - y))
        db = (1/num_samples) * np.sum(y_pred - y)

        # Update weights and bias
        weights -= learning_rate * dw
        bias -= learning_rate * db

    return weights, bias

# Example usage:
# Assuming you have X_train and y_train
X_train = np.array([1, 2, 3, 4, 5])
y_train = np.array([2, 4, 5, 4, 5])

# Reshape X_train to a column vector
X_train = X_train.reshape(-1, 1)

# Fit the model
weights, bias = linear_regression(X_train, y_train)

# Print the coefficients
print("Coefficients:", weights)
print("Intercept (Bias):", bias)


Coefficients: [1.06580687 0.61920125]
Intercept (Bias): 1.065806871244777


In [20]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(Dense(units = 128, input_dim = xtrain.shape[1], activation = 'relu'))
model.add(Dense(units = 64 , activation = 'relu'))
model.add(Dense(units = 1, activation = 'linear'))

model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['mean_absolute_error'])
model.fit(xtrain,ytrain, epochs = 50, batch_size=32, validation_data = (xtest,ytest), verbose = 2)

ypred = model.predict(xtest)

Epoch 1/50
13/13 - 1s - loss: 577.1417 - mean_absolute_error: 22.1834 - val_loss: 527.7573 - val_mean_absolute_error: 21.1074 - 876ms/epoch - 67ms/step
Epoch 2/50
13/13 - 0s - loss: 517.1193 - mean_absolute_error: 20.7564 - val_loss: 461.8546 - val_mean_absolute_error: 19.4667 - 67ms/epoch - 5ms/step
Epoch 3/50
13/13 - 0s - loss: 435.0384 - mean_absolute_error: 18.7074 - val_loss: 366.8901 - val_mean_absolute_error: 16.9262 - 51ms/epoch - 4ms/step
Epoch 4/50
13/13 - 0s - loss: 318.1180 - mean_absolute_error: 15.6072 - val_loss: 247.8832 - val_mean_absolute_error: 13.3132 - 42ms/epoch - 3ms/step
Epoch 5/50
13/13 - 0s - loss: 185.8606 - mean_absolute_error: 11.4467 - val_loss: 140.1546 - val_mean_absolute_error: 9.2094 - 46ms/epoch - 4ms/step
Epoch 6/50
13/13 - 0s - loss: 90.2306 - mean_absolute_error: 7.5339 - val_loss: 89.6051 - val_mean_absolute_error: 6.9656 - 60ms/epoch - 5ms/step
Epoch 7/50
13/13 - 0s - loss: 57.9574 - mean_absolute_error: 5.8474 - val_loss: 71.4383 - val_mean_abso

In [21]:
print('rmse', np.sqrt(mean_squared_error(ytest, ypred)))

rmse 4.738092083477041
